#Mini Project 3 Group 99

## Group Member

Zhenyuan Ma (ID: 260881867)

Yue Xiao (ID: 260740996)

Yihe Zhang (ID: 260738383)

## Connect Google Drive

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


## Preprocess

In [0]:
def imgPreprocess (imgSet):
    modifiedSet = []
    for idx in range(len(imgSet)):
        test = imgSet[idx]
        ret, thresh = cv.threshold(test, 252, 255, 0, cv.THRESH_BINARY)
        ima, contours, hierarchy = cv.findContours(thresh, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
        
        maxC = 0
        maxContour = None
        for contour in contours:
            (x,y,w,h) = cv.boundingRect(contour)
            if max(w,h) > maxC:
                maxC = max(w,h)
                maxContour = contour

        (x,y,w,h) = cv.boundingRect(maxContour)
        modified = np.zeros((64, 64))
        modified[y:y+h+1,x:x+w+1] = thresh[y:y+h+1,x:x+w+1]
        modifiedSet.append(modified)
    
    return np.array(modifiedSet)

In [0]:
import numpy as np
import pandas as pd
import cv2 as cv
from keras import utils as np_utils

Using TensorFlow backend.


In [0]:
# Load Data
df_x = np.array(pd.read_pickle('/content/gdrive/My Drive/Comp551Project3/train_images.pkl'), dtype='uint8')
df_y = pd.read_csv('/content/gdrive/My Drive/Comp551Project3/train_labels.csv').iloc[:, 1]

In [0]:
# Image preprocess
img_x = imgPreprocess(df_x)

# Convert to float32 and reshape the images sets to be [samples][width][height][pixels]
data_x = df_x.reshape(img_x.shape[0], 64, 64, 1).astype('float32')

# Normalize data from 0-255 to 0-1
data_x = data_x / 255

# One hot encode labels sets
data_y = np_utils.to_categorical(df_y)

## Inception v3

###Model Creation

In [0]:
from keras.models import Model
from keras import layers
from keras import backend as K

In [0]:
if K.image_data_format() == 'channels_first':
    channel_axis = 1
else:
    channel_axis = 3

In [0]:
# Utility function to apply convolution and batch normalization.

def conv2d_bn(x,
              filters,
              num_row,
              num_col,
              padding='same',
              strides=(1, 1),
              name=None):
  
    if name is not None:
        bn_name = name + '_bn'
        conv_name = name + '_conv'
    else:
        bn_name = None
        conv_name = None
    if K.image_data_format() == 'channels_first':
        bn_axis = 1
    else:
        bn_axis = 3
    x = layers.Conv2D(
        filters, (num_row, num_col),
        strides=strides,
        padding=padding,
        use_bias=False,
        name=conv_name)(x)
    x = layers.BatchNormalization(axis=bn_axis, scale=False, name=bn_name)(x)
    x = layers.Activation('relu', name=name)(x)
    return x

In [0]:
input_tensor = layers.Input(shape=(64,64,1))

x = conv2d_bn(input_tensor, 4, 3, 3, strides=(1, 1), padding='valid')
x = conv2d_bn(x, 4, 3, 3, padding='valid')
x = conv2d_bn(x, 8, 3, 3, padding='valid')
x = layers.MaxPooling2D((3, 3), strides=(1, 1))(x)

x = conv2d_bn(x, 12, 1, 1, padding='valid')
x = conv2d_bn(x, 32, 3, 3, padding='valid')
x = layers.MaxPooling2D((3, 3), strides=(1, 1))(x)


branch1x1 = conv2d_bn(x, 8, 1, 1,padding='valid')   # 1,1

branch5x5 = conv2d_bn(x, 6, 1, 1,padding='valid') # 1,1
branch5x5 = conv2d_bn(branch5x5, 8, 5, 5) # 5,5 

branch3x3dbl = conv2d_bn(x, 8, 1, 1,padding='valid') # 1,1
branch3x3dbl = conv2d_bn(branch3x3dbl, 12, 3, 3)
branch3x3dbl = conv2d_bn(branch3x3dbl, 12, 3, 3)

branch_pool = layers.AveragePooling2D((3, 3),  strides=(1, 1),  padding='same')(x)
branch_pool = conv2d_bn(branch_pool, 4, 1, 1)
x = layers.concatenate([branch1x1, branch5x5, branch3x3dbl, branch_pool], axis=channel_axis,name='mixed0')


branch3x3 = conv2d_bn(x, 32, 3, 3, strides=(2, 2), padding='valid')

branch3x3dbl = conv2d_bn(x, 8, 1, 1)
branch3x3dbl = conv2d_bn(branch3x3dbl, 12, 3, 3)
branch3x3dbl = conv2d_bn(
    branch3x3dbl, 12, 3, 3, strides=(2, 2), padding='valid')

branch_pool = layers.MaxPooling2D((3, 3), strides=(2, 2))(x)
x = layers.concatenate(
    [branch3x3, branch3x3dbl, branch_pool],
    axis=channel_axis,
    name='mixed1')


branch1x1 = conv2d_bn(x, 32, 1, 1,padding='valid') # 1 1

branch7x7 = conv2d_bn(x, 16, 1, 1,padding='valid') # 1 1
branch7x7 = conv2d_bn(branch7x7, 16, 1, 7)
branch7x7 = conv2d_bn(branch7x7, 32, 7, 1)

branch7x7dbl = conv2d_bn(x, 16, 1, 1,padding='valid') # 1 1
branch7x7dbl = conv2d_bn(branch7x7dbl, 16, 7, 1)
branch7x7dbl = conv2d_bn(branch7x7dbl, 16, 1, 7)
branch7x7dbl = conv2d_bn(branch7x7dbl, 16, 7, 1)
branch7x7dbl = conv2d_bn(branch7x7dbl, 32, 1, 7)

branch_pool = layers.AveragePooling2D((3, 3),
                                      strides=(1, 1 ),
                                      padding='same')(x)
branch_pool = conv2d_bn(branch_pool, 32, 1, 1,padding='valid') # 1 1
x = layers.concatenate(
    [branch1x1, branch7x7, branch7x7dbl, branch_pool],
    axis=channel_axis,
    name='mixed2')


branch3x3 = conv2d_bn(x, 32, 1, 1,padding='valid') # 1 1
branch3x3 = conv2d_bn(branch3x3, 64, 3, 3,
                      strides=(2, 2), padding='valid')

branch7x7x3 = conv2d_bn(x, 32, 1, 1,padding='valid') # 1 1
branch7x7x3 = conv2d_bn(branch7x7x3, 32, 1, 7)
branch7x7x3 = conv2d_bn(branch7x7x3, 32, 7, 1)
branch7x7x3 = conv2d_bn(
    branch7x7x3, 32, 3, 3, strides=(2, 2), padding='valid')

branch_pool = layers.MaxPooling2D((3, 3), strides=(2, 2),padding='valid')(x) # strides 2 2
x = layers.concatenate(
    [branch3x3, branch7x7x3, branch_pool],
    axis=channel_axis,
    name='mixed3')



branch1x1 = conv2d_bn(x, 32, 1, 1,padding='valid',name='validcheck') # 1 1

branch3x3 = conv2d_bn(x, 32, 1, 1,padding='valid')
branch3x3_1 = conv2d_bn(branch3x3, 32, 1, 3)
branch3x3_2 = conv2d_bn(branch3x3, 32, 3, 1)
branch3x3 = layers.concatenate(
    [branch3x3_1, branch3x3_2],
    axis=channel_axis,
    name='mixed4')

branch3x3dbl = conv2d_bn(x, 32, 1, 1,padding='valid')
branch3x3dbl = conv2d_bn(branch3x3dbl, 32, 3, 3)
branch3x3dbl_1 = conv2d_bn(branch3x3dbl, 32, 1, 3)
branch3x3dbl_2 = conv2d_bn(branch3x3dbl, 32, 3, 1)
branch3x3dbl = layers.concatenate(
    [branch3x3dbl_1, branch3x3dbl_2], axis=channel_axis)

branch_pool = layers.AveragePooling2D(
    (3, 3), strides=(1, 1), padding='same')(x)
branch_pool = conv2d_bn(branch_pool, 32, 1, 1,padding='valid') # 1 1 
x = layers.concatenate(
    [branch1x1, branch3x3, branch3x3dbl, branch_pool],
    axis=channel_axis,
    name='mixed5')
        
      
x = layers.AveragePooling2D(
    (3, 3), strides=(1, 1), padding='valid')(x)
x = layers.Dropout(0.01)(x)
x = layers.GlobalAveragePooling2D(name='avg_pool')(x)
x = layers.Dense(64, activation = "relu")(x)
x = layers.Dense(10, activation='softmax', name='predictions')(x)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
model = Model(input_tensor, x)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
# Fiting the training set to the model, with epochs = 10
modelfit = model.fit(x=data_x, y=data_y, batch_size=100, epochs=10, verbose=1, callbacks=None, validation_split=0.1, shuffle=True, class_weight=None, sample_weight=None, initial_epoch=0, steps_per_epoch=None, validation_steps=None)

Train on 36000 samples, validate on 4000 samples
Epoch 1/10
36000/36000 [==============================] - 79s 2ms/step - loss: 1.1247 - acc: 0.6328 - val_loss: 2.4216 - val_acc: 0.4382
Epoch 2/10
36000/36000 [==============================] - 79s 2ms/step - loss: 0.4467 - acc: 0.8719 - val_loss: 0.5125 - val_acc: 0.8477
Epoch 3/10
36000/36000 [==============================] - 78s 2ms/step - loss: 0.3327 - acc: 0.9057 - val_loss: 0.9342 - val_acc: 0.7412
Epoch 4/10
36000/36000 [==============================] - 79s 2ms/step - loss: 0.2717 - acc: 0.9199 - val_loss: 0.4579 - val_acc: 0.8727
Epoch 5/10
36000/36000 [==============================] - 78s 2ms/step - loss: 0.2324 - acc: 0.9319 - val_loss: 1.1392 - val_acc: 0.6820
Epoch 6/10
36000/36000 [==============================] - 79s 2ms/step - loss: 0.2012 - acc: 0.9407 - val_loss: 0.4275 - val_acc: 0.8800
Epoch 7/10
36000/36000 [==============================] - 78s 2ms/step - loss: 0.1792 - acc: 0.9456 - val_loss: 0.4552 - val_acc:

###Data Augmentation

In [0]:
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator

In [0]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=15,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.15, # Randomly zoom image 
        width_shift_range=0.15,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.15,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images

In [0]:
datagen.fit(data_x)

In [0]:
# Use cross validation
n_splits = 26
for _ in range(n_splits):
	x_train, x_val, y_train, y_val = train_test_split(data_x, data_y, test_size=0.10)
	# Images augmentation
	modelfitwithdatagen = model.fit_generator(datagen.flow(x_train, y_train, batch_size=100),
                              validation_data=(x_val, y_val),                             
                              epochs=5, verbose = 1, steps_per_epoch=1000)

Epoch 1/5
1000/1000 [==============================] - 218s 218ms/step - loss: 0.4679 - acc: 0.8593 - val_loss: 0.3998 - val_acc: 0.8758
Epoch 2/5
1000/1000 [==============================] - 218s 218ms/step - loss: 0.3897 - acc: 0.8800 - val_loss: 0.2605 - val_acc: 0.9168
Epoch 3/5
1000/1000 [==============================] - 217s 217ms/step - loss: 0.3472 - acc: 0.8925 - val_loss: 0.2153 - val_acc: 0.9357
Epoch 4/5
1000/1000 [==============================] - 217s 217ms/step - loss: 0.3218 - acc: 0.9001 - val_loss: 0.1832 - val_acc: 0.9468
Epoch 5/5
1000/1000 [==============================] - 216s 216ms/step - loss: 0.3056 - acc: 0.9058 - val_loss: 0.2006 - val_acc: 0.9443
Epoch 1/5
1000/1000 [==============================] - 216s 216ms/step - loss: 0.2923 - acc: 0.9095 - val_loss: 0.1451 - val_acc: 0.9543
Epoch 2/5
1000/1000 [==============================] - 217s 217ms/step - loss: 0.2748 - acc: 0.9149 - val_loss: 0.2776 - val_acc: 0.9232
Epoch 3/5
1000/1000 [====================

### Output

In [0]:
# Load and reshape the test iamges
df_test = np.array(pd.read_pickle('/content/gdrive/My Drive/Comp551Project3/test_images.pkl'), dtype='uint8')
data_test = df_test.reshape(df_test.shape[0], 64, 64, 1).astype('float32')
# Normalize test data from 0-255 to 0-1
x_test = data_test / 255

In [0]:
# Predict lables
pred = model.predict(x_test)
pred_val = np.argmax(pred,axis=1)

In [0]:
# Write
pd.DataFrame(pred_val, columns=['Category']).to_csv("./submission_36000_140.csv", index_label='Id')
pd.DataFrame(pred_val, columns=['Category']).to_csv("/content/gdrive/My Drive/submission_36000_140.csv", index_label='Id')

##CNN

In [0]:
import numpy
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import Lambda
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.callbacks import ReduceLROnPlateau
from keras.utils import np_utils
from keras import backend as K
K.set_image_dim_ordering('th')
from keras.preprocessing import image
gen = image.ImageDataGenerator()

### Define the model

In [0]:
# reshape to be [samples][pixels][width][height]
data2_x = df_x.reshape(img_x.shape[0], 1, 64, 64).astype('float32')

# Normalize data from 0-255 to 0-1
data2_x = data2_x / 255

# Split data
X_train = data2_x[:32000]
Y_train = data_y[:32000]

X_validation = data2_x[32000:]
Y_validation = data_y[32000:]

num_classes = Y_validation.shape[1]

batches = gen.flow(X_train, Y_train)
val_batches = gen.flow(X_validation, Y_validation)

In [0]:
# define the model
def model():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(5, 5),activation='relu',kernel_initializer='he_normal',input_shape=(1, 64, 64)))
    model.add(Conv2D(32, kernel_size=(5, 5),activation='relu',kernel_initializer='he_normal'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.20))
    model.add(Conv2D(64, (3, 3), activation='relu',padding='same',kernel_initializer='he_normal'))
    model.add(Conv2D(64, (3, 3), activation='relu',padding='same',kernel_initializer='he_normal'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
    model.add(Dropout(0.25))
    model.add(Conv2D(128, (3, 3), activation='relu',padding='same',kernel_initializer='he_normal'))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.25))
    model.add(Dense(num_classes, activation='softmax'))
    
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.RMSprop(), metrics=['accuracy'])
    return model

In [0]:
# build the model
model = model()

450 steps per epoch & 50 validation steps

26 epochs

In [0]:
model.fit_generator(generator=batches, steps_per_epoch=450, epochs=2,
               validation_data=val_batches, validation_steps=50)

In [0]:
# Final evaluation of the model
scores = model.evaluate(X_validation, Y_validation, verbose=0)
print("Prediction: %.2f%%" % (scores[1]*100))